In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128 # 64 - 32
num_classes = 10
epochs = 50
l = 40
num_filter = 24 # 12
compression = 1 # 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 24
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)

In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [13]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 368s 7ms/step - loss: 1.4445 - acc: 0.4676 - val_loss: 1.5907 - val_acc: 0.4912
Epoch 2/50
19200/50000 [==========>...................] - ETA: 3:20 - loss: 1.0665 - acc: 0.6139

50000/50000 [==============================] - 350s 7ms/step - loss: 0.9789 - acc: 0.6491 - val_loss: 5.7558 - val_acc: 0.3001
Epoch 3/50
45440/50000 [==========================>...] - ETA: 29s - loss: 0.7925 - acc: 0.7186

50000/50000 [==============================] - 350s 7ms/step - loss: 0.7885 - acc: 0.7205 - val_loss: 1.0543 - val_acc: 0.6755
Epoch 4/50
50000/50000 [==============================] - 351s 7ms/step - loss: 0.6806 - acc: 0.7590 - val_loss: 0.8427 - val_acc: 0.7255
Epoch 5/50
 3200/50000 [>.............................] - ETA: 5:05 - loss: 0.6218 - acc: 0.7863

50000/50000 [==============================] - 350s 7ms/step - loss: 0.6032 - acc: 0.7882 - val_loss: 1.2131 - val_acc: 0.6656
Epoch 6/50
39424/50000 [======================>.......] - ETA: 1:08 - loss: 0.5484 - acc: 0.8099

50000/50000 [==============================] - 350s 7ms/step - loss: 0.5434 - acc: 0.8118 - val_loss: 0.9206 - val_acc: 0.7350
Epoch 7/50
50000/50000 [==============================] - 351s 7ms/step - loss: 0.5012 - acc: 0.8245 - val_loss: 1.4477 - val_acc: 0.6250
Epoch 8/50
 1792/50000 [>.............................] - ETA: 5:14 - loss: 0.4724 - acc: 0.8449

...

50000/50000 [==============================] - 349s 7ms/step - loss: 0.0811 - acc: 0.9708 - val_loss: 0.6622 - val_acc: 0.8675
Epoch 45/50
38528/50000 [======================>.......] - ETA: 1:14 - loss: 0.0835 - acc: 0.9685

50000/50000 [==============================] - 349s 7ms/step - loss: 0.0830 - acc: 0.9691 - val_loss: 0.7700 - val_acc: 0.8500
Epoch 46/50
50000/50000 [==============================] - 349s 7ms/step - loss: 0.0766 - acc: 0.9723 - val_loss: 0.8157 - val_acc: 0.8410
Epoch 47/50
 1408/50000 [..............................] - ETA: 5:14 - loss: 0.0889 - acc: 0.9666

50000/50000 [==============================] - 349s 7ms/step - loss: 0.0772 - acc: 0.9728 - val_loss: 0.6422 - val_acc: 0.8697
Epoch 48/50
38528/50000 [======================>.......] - ETA: 1:14 - loss: 0.0704 - acc: 0.9753

50000/50000 [==============================] - 349s 7ms/step - loss: 0.0734 - acc: 0.9739 - val_loss: 0.8195 - val_acc: 0.8466
Epoch 49/50
50000/50000 [==============================] - 349s 7ms/step - loss: 0.0738 - acc: 0.9739 - val_loss: 0.7005 - val_acc: 0.8637
Epoch 50/50
 1408/50000 [..............................] - ETA: 5:15 - loss: 0.0651 - acc: 0.9737

50000/50000 [==============================] - 349s 7ms/step - loss: 0.0722 - acc: 0.9743 - val_loss: 0.8659 - val_acc: 0.8451


In [14]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 32s 3ms/step
Test loss: 0.8659206039965153
Test accuracy: 0.8451


In [16]:
# Save the trained weights in to .h5 format
model.save_weights("HARDAYAL_BATCH_5_ASSIGNMENT4B_0.8451.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('HARDAYAL_BATCH_5_ASSIGNMENT4B_0.8451.h5')